In [1]:
import sys

In [5]:
!{sys.executable} -m pip install biopython GEOparse biomart nltk

distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 1.5 MB 264 kB/s eta 0:00:01
     |████████████████████████████████| 772 kB 2.5 MB/s eta 0:00:01
distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None


In [90]:
from Bio import Entrez
import GEOparse
import pandas as pd
import re
import biomart
import nltk
from nltk.tokenize import word_tokenize  
import json
from os import path
import json

In [91]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
Entrez.email = "emerson@snthesis.com"

In [85]:
def get_ensembl_mappings():                                                    
    server = biomart.BiomartServer('http://www.ensembl.org/biomart', verbose=True)         
    mart = server.datasets['hsapiens_gene_ensembl']  
    
    attributes = [
    "affy_hg_u133_plus_2",
    "ensembl_gene_id",
    "entrezgene_id",
    "external_gene_name"]
                                                                                
    # Get the mapping between the attributes                                    
    response = mart.search({'attributes': attributes})                          
    data = response.raw.data.decode('ascii')                                    
                                                                                
    ensembl_to_genesymbol = {}                                                  
    # Store the data in a dict                                                  
    for line in data.splitlines():                                              
        line = line.split('\t')                                                 
        # The entries are in the same order as in the `attributes` variable
        probe_id = line[0]                                                 
        ensembl_gene_id = line[1]                                                   
        entrez_id = line[2]                                                  
        external_gene_name = line[3]                                               
                                                                                
        # Some of these keys may be an empty string. If you want, you can 
        # avoid having a '' key in your dict by ensuring the 
        # transcript/gene/peptide ids have a nonzero length before
        # adding them to the dict
        ensembl_to_genesymbol[probe_id] = entrez_id                      
        ensembl_to_genesymbol[ensembl_gene_id] = entrez_id                       
        ensembl_to_genesymbol[external_gene_name] = entrez_id                
                                                                                
    return ensembl_to_genesymbol

In [86]:
if not path.exists('./probe_mappings_affy_hg_u133_plus_2.json'):
    probe_mappings = get_ensembl_mappings()
    with open('./probe_mappings_affy_hg_u133_plus_2.json', 'w') as file:
        file.write(json.dumps(mappings))
else:
    with open('./probe_mappings_affy_hg_u133_plus_2.json') as file:
        probe_mappings = json.load(file)

[BiomartServer:'http://www.ensembl.org/biomart/martservice'] is alive.
[BiomartServer:'http://www.ensembl.org/biomart/martservice'] Fetching datasets
[BiomartServer:'http://www.ensembl.org/biomart/martservice'] Fetching databases
[BiomartDatabase:'Ensembl Genes 108'] Fetching datasets
[BiomartDatabase:'Mouse strains 108'] Fetching datasets
[BiomartDatabase:'Sequence'] Fetching datasets
[BiomartDatabase:'Ontology'] Fetching datasets
[BiomartDatabase:'Genomic features 108'] Fetching datasets
[BiomartDatabase:'Ensembl Variation 108'] Fetching datasets
[BiomartDatabase:'Ensembl Regulation 108'] Fetching datasets
[BiomartDataset:'hsapiens_gene_ensembl'] Searching using following params:
{'attributes': ['affy_hg_u133_plus_2',
                'ensembl_gene_id',
                'entrezgene_id',
                'external_gene_name']}
[BiomartDataset:'hsapiens_gene_ensembl'] Fetching attributes
[BiomartDataset] search query:
b'<Query virtualSchemaName="default" formatter="TSV" header="0" uniqueR

In [23]:
series = 'GSE72970'
gse = GEOparse.get_GEO(geo=series, destdir="./geo_data")

09-Nov-2022 20:00:36 DEBUG utils - Directory ./geo_data already exists. Skipping.
09-Nov-2022 20:00:36 INFO GEOparse - File already exist: using local version.
09-Nov-2022 20:00:36 INFO GEOparse - Parsing ./geo_data/GSE72970_family.soft.gz: 
09-Nov-2022 20:00:36 DEBUG GEOparse - DATABASE: GeoMiame
09-Nov-2022 20:00:36 DEBUG GEOparse - SERIES: GSE72970
09-Nov-2022 20:00:36 DEBUG GEOparse - PLATFORM: GPL570
/opt/conda/lib/python3.8/site-packages/GEOparse/GEOparse.py:113: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return parse_GSE(filepath, open_kwargs=open_kwargs)
09-Nov-2022 20:00:37 DEBUG GEOparse - SAMPLE: GSM1875897
09-Nov-2022 20:00:37 DEBUG GEOparse - SAMPLE: GSM1875898
09-Nov-2022 20:00:37 DEBUG GEOparse - SAMPLE: GSM1875899
09-Nov-2022 20:00:37 DEBUG GEOparse - SAMPLE: GSM1875900
09-Nov-2022 20:00:37 DEBUG GEOparse - SAMPLE: GSM1875901
09-Nov-2022 20:00:37 DEBUG GEOparse - SAMPLE: GSM1875902
09-Nov-2022 20:00:37 DEBUG GEOp

In [24]:
short = [(name, gsm) for index, (name, gsm) in enumerate(gse.gsms.items()) if index < 3]

In [25]:
sample = short[0][1]
dir(sample)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__metaclass__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_columns_as_string',
 '_get_metadata_as_string',
 '_get_object_as_soft',
 '_get_table_as_string',
 'annotate',
 'annotate_and_average',
 'columns',
 'download_SRA',
 'download_supplementary_files',
 'geotype',
 'get_accession',
 'get_metadata_attribute',
 'get_type',
 'head',
 'metadata',
 'name',
 'relations',
 'show_columns',
 'show_metadata',
 'show_table',
 'table',
 'to_soft']

In [26]:
sample.metadata

{'title': ['Primary_Tumor 01_010'],
 'geo_accession': ['GSM1875897'],
 'status': ['Public on Aug 17 2017'],
 'submission_date': ['Sep 12 2015'],
 'last_update_date': ['Aug 17 2017'],
 'type': ['RNA'],
 'channel_count': ['1'],
 'source_name_ch1': ['Tissue samples taken from colorectal cancer patients'],
 'organism_ch1': ['Homo sapiens'],
 'taxid_ch1': ['9606'],
 'characteristics_ch1': ['center: Tours',
  'Sex: Male',
  'age: 53',
  'who performance status: 0',
  'tumor location: Left colon',
  'synchronous metastase: Yes',
  'pn: pN2',
  'pt: pT3',
  'regimen: FOLFOX',
  'response category: PR',
  'response status: R',
  'pfs censored: 1',
  'pfs: 5.585215569',
  'os censored: 1',
  'os: 5.585215569'],
 'treatment_protocol_ch1': ['Samples were collected before chemotherapy'],
 'molecule_ch1': ['total RNA'],
 'extract_protocol_ch1': ["Rneasy kit (Qiagen) for total RNA was performed according to the manufacturer's instructions."],
 'label_ch1': ['biotin'],
 'label_protocol_ch1': ['Biotiny

In [70]:
sdf = sample.table

In [75]:
len(sdf['ID_REF'].unique())

54675

In [76]:
def probe_to_entrez_id(probe_id, probe_mappings):
    mapped = probe_mappings.get(probe_id, probe_id)
    if mapped is None or mapped.strip() == '':
        return probe_id
    return mapped

def to_expression_matrix(dataset, probe_mappings):
    items = {item_id: sample.table['VALUE'].tolist() for item_id, sample in dataset.gsms.items()}
    item = [sample for i, (_, sample) in enumerate(dataset.gsms.items()) if i < 1][0]
    labels = [probe_to_entrez_id(probe_id, probe_mappingsprobe_to_entrez_id) for probe_id in item.table['ID_REF']]
    len_labels = len(labels)
    items['entrez_id'] = labels
    return pd.DataFrame.from_dict(items)

In [30]:
def safe_get(data, path_item, default={}):
    if isinstance(data, dict):
        return data.get(path_item, default)
    else:
        length = len(data)
        return data[path_item] if path_item < length else default
    

def path_mapping(path):
    def mapper(data):
        for path_item in path[:-1]:
            data = safe_get(data, path_item)
        return safe_get(data, path[-1], None)
    return mapper


def value(value):
    return lambda _data: value

In [31]:
units = set([
    "months"
])
responses = {
    "r": "response",
}
response_categories = {
    "pr": "partial_response",
    "cr": "complete_response",
    "sd": "stable_disease",
    "pd": "progressive_disease"
}
sample_collected = set(["before_treatment", "after_treatment"])

In [32]:
headers = ["sample_id", "sample_id_type", "source_accession", "source_type", "sex", "age", "normalized_drug", "raw_drug", "sample_name", "data_processing_notes", "sample_collected", "response", "response_category", "progression_free_survival", "progression_free_survival_unit", "overall_survival", "overall_survival_unit", "platform_id", 'tmn_t', 'tmn_n', 'tmn_m']

In [94]:
with open('./drug_synonyms.json') as file:
    syn_json = json.load(file)
    
drop_tokens = set([
    'study drug'
])

In [96]:
def normalize_drug_name(drug_name):
    drugs = re.split(r"\+|,|/", drug_name)
    for dt in drop_tokens:
        drugs = [d.replace(dt, '') for d in drugs]
    drugs = [re.sub(r"(\d)([A-z])", "\\1 \\2", d.lower().strip()) for d in drugs]
    drugs = [re.sub(r"(\d)\s*-\s*([A-z])", "\\1 \\2", d) for d in drugs]
    tokens = [" ".join(word_tokenize(d)) for d in drugs if d not in drop_tokens]
    tokens = [syn_json['synonyms'].get(t, t) for t in tokens]
    tokens.sort()
    return "|".join(tokens)

In [97]:
def normalize_tmn_n(raw):
    return re.sub(r'.*N?(\d|X).*', 'N\\1', raw.upper())

def normalize_tmn_t(raw):
    return re.sub(r'.*T?(\d|X).*', 'T\\1', raw.upper())

In [98]:
mapping_GSE72970 = {
    "sex": path_mapping(["characteristics_ch1", "sex"]),
    "age": path_mapping(["characteristics_ch1", "age"]),
    "raw_drug": path_mapping(["characteristics_ch1", "regimen"]),
    "response": path_mapping(["characteristics_ch1", "response status"]),
    'response_category': path_mapping(["characteristics_ch1", "response category"]),
    'progression_free_survival': path_mapping(["characteristics_ch1", "pfs"]),
    "progression_free_survival_unit": value("months"),
    'overall_survival': path_mapping(["characteristics_ch1", 'os']),
    "overall_survival_unit": value("months"),
    "sample_name": path_mapping(["title", 0]),
    "data_processing_notes": path_mapping(["data_processing", 0]),
    "sample_collected": "before_treatment",
    "sample_id": path_mapping(['geo_accession', 0]),
    'sample_id_type': "geo_accession",
    "normalized_drug": lambda data: normalize_drug_name(path_mapping(["characteristics_ch1", "regimen"])(data)),
    "source_accession": path_mapping(['series_id', -1]),
    "source_type": "geo_series",
    "platform_id": path_mapping(['platform_id', 0]),
    'tmn_n': lambda data: normalize_tmn_n(path_mapping(["characteristics_ch1", 'pn'])(data)),
    'tmn_t': lambda data: normalize_tmn_t(path_mapping(["characteristics_ch1", 'pt'])(data)),
    'tmn_m': None
}

def normalize_gse_from_GSE72970(metadata):
    new_metadata = dict(metadata)
    raw_characteristics = metadata.get('characteristics_ch1', [])
    characteristics = {}
    for item in raw_characteristics:
        parts = item.split(':', 1)
        key = parts[0].strip().lower()
        value = parts[1].strip()
        characteristics[key] = value
    new_metadata['characteristics_ch1'] = characteristics
    
    return new_metadata

In [38]:
normalize_gse_from_GSE72970(short[0][1].metadata)

{'title': ['Primary_Tumor 01_010'],
 'geo_accession': ['GSM1875897'],
 'status': ['Public on Aug 17 2017'],
 'submission_date': ['Sep 12 2015'],
 'last_update_date': ['Aug 17 2017'],
 'type': ['RNA'],
 'channel_count': ['1'],
 'source_name_ch1': ['Tissue samples taken from colorectal cancer patients'],
 'organism_ch1': ['Homo sapiens'],
 'taxid_ch1': ['9606'],
 'characteristics_ch1': {'center': 'Tours',
  'sex': 'Male',
  'age': '53',
  'who performance status': '0',
  'tumor location': 'Left colon',
  'synchronous metastase': 'Yes',
  'pn': 'pN2',
  'pt': 'pT3',
  'regimen': 'FOLFOX',
  'response category': 'PR',
  'response status': 'R',
  'pfs censored': '1',
  'pfs': '5.585215569',
  'os censored': '1',
  'os': '5.585215569'},
 'treatment_protocol_ch1': ['Samples were collected before chemotherapy'],
 'molecule_ch1': ['total RNA'],
 'extract_protocol_ch1': ["Rneasy kit (Qiagen) for total RNA was performed according to the manufacturer's instructions."],
 'label_ch1': ['biotin'],
 '

In [99]:
def map_sample(sample, headers, mapping, normalizer):
    data = {}
    metadata = normalizer(sample.metadata)
    for header in headers:
        mapper = mapping.get(header)
        
        if callable(mapper):
            data[header] = mapper(metadata)
        else:
            data[header] = mapper
    return data

def make_gse_sample_records(gse_data, headers, mapping, normalizer):
    return [map_sample(sample, headers, mapping, normalizer) for _name, sample in gse_data.gsms.items()]

In [100]:
GSE72970_meta = pd.DataFrame.from_records(make_gse_sample_records(gse, headers, mapping_GSE72970, normalize_gse_from_GSE72970))
GSE72970_meta

,sample_id,sample_id_type,source_accession,source_type,sex,age,normalized_drug,raw_drug,sample_name,data_processing_notes,...,response,response_category,progression_free_survival,progression_free_survival_unit,overall_survival,overall_survival_unit,platform_id,tmn_t,tmn_n,tmn_m
0,GSM1875897,geo_accession,GSE72970,geo_series,Male,53,fluorouracil|leucovorin|oxaliplatin,FOLFOX,Primary_Tumor 01_010,Processing and normalization was done using th...,...,R,PR,5.585215569,months,5.585215569,months,GPL570,T3,N2,None
1,GSM1875898,geo_accession,GSE72970,geo_series,Male,57,fluorouracil|leucovorin|oxaliplatin,FOLFOX,Primary_Tumor 02_005,Processing and normalization was done using th...,...,R,PR,8.47638607,months,13.66735077,months,GPL570,T3,N2,None
2,GSM1875899,geo_accession,GSE72970,geo_series,Female,55,fluorouracil|leucovorin|oxaliplatin,FOLFOX,Primary_Tumor 02_006,Processing and normalization was done using th...,...,NR,PD,1.872689962,months,2.49691987,months,GPL570,T4,N2,None
3,GSM1875900,geo_accession,GSE72970,geo_series,Female,66,fluorouracil|leucovorin|oxaliplatin,FOLFOX,Primary_Tumor 02_007,Processing and normalization was done using th...,...,R,PR,70.01232147,months,70.01232147,months,GPL570,T3,N1,None
4,GSM1875901,geo_accession,GSE72970,geo_series,Female,56,fluorouracil|leucovorin|irinotecan,FOLFIRI,Primary_Tumor 02_008,Processing and normalization was done using th...,...,R,PR,7.55646801,months,19.94250488,months,GPL570,T4,N2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,GSM1876016,geo_accession,GSE72970,geo_series,Female,75,bevacizumab|fluorouracil|leucovorin|irinotecan,FOLFIRI+BEVACIZUMAB,Primary_Tumor 8133,Processing and normalization was done using th...,...,R,PR,22.30800819,months,28.35318184,months,GPL570,T3,N2,None
120,GSM1876017,geo_accession,GSE72970,geo_series,Female,61,bevacizumab|fluorouracil|leucovorin|irinotecan,FOLFIRI+BEVACIZUMAB,Primary_Tumor 8135,Processing and normalization was done using th...,...,NR,SD,4.43531847,months,47.86858368,months,GPL570,T4,N2,None
121,GSM1876018,geo_accession,GSE72970,geo_series,Male,66,bevacizumab|fluorouracil|leucovorin|irinotecan,FOLFIRI+BEVACIZUMAB,Primary_Tumor 8141,Processing and normalization was done using th...,...,NR,SD,7.392197132,months,7.392197132,months,GPL570,T3,N2,None
122,GSM1876019,geo_accession,GSE72970,geo_series,Female,54,bevacizumab|fluorouracil|leucovorin|irinotecan...,FOLFIRINOX+BEVACIZUMAB,Primary_Tumor 8170,Processing and normalization was done using th...,...,R,PR,25.72484589,months,39.78644943,months,GPL570,T3,N2,None


In [101]:
GSE72970_expression = to_expression_matrix(gse, probe_mappings)

In [102]:
GSE72970_expression.to_csv('./GSE72970_expression_matrix_raw.csv', index=False)

In [103]:
GSE72970_meta.to_csv('./GSE72970_sample_metadata_raw.csv', index=False)